Summary of available models:

- yolov8n (Done)
- yolov8s (Done)
- yolov8m (Done)
- yolov8l (Done)
- yolov8x (Done)
- yolov5n (Done)
- yolov5s (Done)
- yolov5m (Done)
- yolov5l (Done)
- yolov5x (done)
- yolov3 (Done)
- yolov3u (Done)
- yolov3-spp (Done)
- yolov3-tiny (Done)

For More Epoch :
- yolov3u (100) (Done)
- yolov5x (100) (Failed (No Runtime exceed limit)) - Res 75
- yolov8x (100) (No Computing Unit available) - Res 75

In [5]:
import os

# Define paths for local environment
local_base_path = "./"  # Current directory
yaml_path = os.path.join(local_base_path, "Dataset", "YOLODatasetFull", "dataset.yaml")  # Path to your YAML file
results_dir = os.path.join(local_base_path, "results")       # Directory to save results
os.makedirs(results_dir, exist_ok=True)

# Print the paths to confirm
print(f"YAML Path: {yaml_path}")
print(f"Results Directory: {results_dir}")

# **Important Considerations:**

# 1.  **Dataset Location:**
#     * Ensure that your dataset is located in the  `./Dataset/YOLODatasetFull`  directory, relative to where you run this Python script.  If your dataset is in a different location, you'll need to adjust the  `local_base_path`  or  `yaml_path`  variables accordingly.
#     * The script assumes your `dataset.yaml` is inside the `YOLODatasetFull` folder.
#
# 2.  **YAML File Content:**
#     * The `dataset.yaml` file should contain the correct paths to your training and validation data.  If these paths are absolute paths (e.g.,  `/content/drive/...`), you'll need to change them to relative paths that are valid in your local environment.
#
# 3. **results directory:**
#     * The results directory is now created at the root level of your project

# Example `dataset.yaml` structure (check and modify as needed):
# ```yaml
# train: ./Dataset/YOLODatasetFull/train/images
# val: ./Dataset/YOLODatasetFull/valid/images
# test: ./Dataset/YOLODatasetFull/test/images  # If you have a test set
# nc: 80  # Number of classes
# names: ['class1', 'class2', ..., 'class80']  # Class names
# ```


YAML Path: ./Dataset\YOLODatasetFull\dataset.yaml
Results Directory: ./results


In [6]:
import os
from ultralytics import YOLO
from tqdm import tqdm
from datetime import datetime
import time

# Paths
local_base_path = "./"  # Current directory
yaml_path = os.path.join(local_base_path, "Dataset", "YOLODatasetFull", "dataset.yaml")
results_dir = os.path.join(local_base_path, "results")
os.makedirs(results_dir, exist_ok=True)

#epoch = 12
#epoch = 100
# epoch = 75
epoch = 50

# imagesize = 640
imagesize = 512

# batchs = 20 # Maximum For Yolo v3 (14,8 GB Peak)
# batchs = 16 # Maximum For Yolo v5x (14,8 GB Peak)

# Augmented

Only Do Augmented Once Only, and dont forget to Save your original Dataset

In [ ]:
# import os
# import random
# import cv2
# import albumentations as A
# from collections import defaultdict

# image_dir = "./Dataset/YOLODatasetFull/images/train"
# label_dir = "./Dataset/YOLODatasetFull/labels/train"

# augment = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.RandomBrightnessContrast(p=0.5),
#     A.GaussNoise(p=0.3),
#     A.Rotate(limit=10, p=0.5)
# ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# # Count samples per class
# class_counts = defaultdict(int)
# label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
# for lf in label_files:
#     with open(os.path.join(label_dir, lf), 'r') as f:
#         for line in f:
#             class_id = line.strip().split()[0]
#             class_counts[class_id] += 1

# if not class_counts:
#     raise ValueError("No class labels found. Check if your label files are formatted correctly.")

# min_count = min(class_counts.values())
# target_count = max(min_count * 2, 20)

# for class_id, count in class_counts.items():
#     if count >= target_count:
#         continue
#     needed = target_count - count
#     print(f"Augmenting class {class_id}: need {needed} more samples")
#     matching_files = []
#     for lf in label_files:
#         with open(os.path.join(label_dir, lf), 'r') as f:
#             for line in f:
#                 if line.strip().split()[0] == class_id:
#                     matching_files.append(lf)
#                     break

#     for i in range(needed):
#         src_label = random.choice(matching_files)
#         base_name = src_label.replace('.txt', '')
#         img_path_jpg = os.path.join(image_dir, base_name + '.jpg')
#         img_path_png = os.path.join(image_dir, base_name + '.png')

#         if os.path.exists(img_path_jpg):
#             img_path = img_path_jpg
#             img_ext = '.jpg'
#         elif os.path.exists(img_path_png):
#             img_path = img_path_png
#             img_ext = '.png'
#         else:
#             print(f"Image not found for label: {src_label}")
#             continue
#         lbl_path = os.path.join(label_dir, src_label)

#         image = cv2.imread(img_path)
#         if image is None:
#             continue
#         with open(lbl_path, 'r') as f:
#             lines = f.readlines()
#         boxes = []
#         classes = []
#         for line in lines:
#             parts = line.strip().split()
#             if parts[0] == class_id:
#                 x, y, bw, bh = map(float, parts[1:])
#                 boxes.append([x, y, bw, bh])
#                 classes.append(class_id)

#         if not boxes:
#             continue
#         augmented = augment(image=image, bboxes=boxes, class_labels=classes)
#         aug_img = augmented['image']
#         aug_boxes = augmented['bboxes']

#         #save_img_name = src_image.replace('.jpg', f'_aug{i}.jpg')
#         save_img_name = base_name + f'_aug{i}{img_ext}'
#         save_lbl_name = src_label.replace('.txt', f'_aug{i}.txt')
#         cv2.imwrite(os.path.join(image_dir, save_img_name), aug_img)
#         with open(os.path.join(label_dir, save_lbl_name), 'w') as f:
#             for box in aug_boxes:
#                 f.write(f"{class_id} {' '.join([f'{v:.6f}' for v in box])}\n")


# YOLOv



In [8]:
# Metrics storage
metrics = {}

# Remmeber to change the batch !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# batch01 = 4 # Reccomended Low for 2050 RTX 4GB YOLOv3
batch01 = 8 # Reccomended High for 2050 RTX 4GB
# batch01 = 12 # High for 2050 RTX 4GB

# Model to train and evaluate
model_name = "YOLOv8m"       # Change this to the model you want to train (e.g., YOLOv4, YOLOv5x)
model_path = "yolov8m.pt"    # Pretrained model path for YOLOv3 (update as needed)

# Generate unique directory name based on model and current time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
results_dirs = os.path.join(results_dir, f"{current_time}, {model_name} with E-B {epoch}-{batch01}")
os.makedirs(results_dirs, exist_ok=True)  # Create the directory if it doesn't exist

# Train and Evaluate the Model
print(f"\nTraining and evaluating {model_name}...")

# Start time tracking
start_time = time.time()

# Train the model
model = YOLO(model_path)  # Load pre-trained model
model.train(
    task="detect",
    data=yaml_path,
    epochs=epoch,  # Set epochs for quick testing; increase for better results
    imgsz=imagesize,
    batch=batch01,  # Adjust to fit your GPU memory
    name=f"{model_name}_detection",
    save_dir=os.path.join(results_dirs, model_name)
)
# End time tracking
end_time = time.time()
training_duration_hours = (end_time - start_time) / 3600  # Convert seconds to hours
print(f"Model and results saved to: {results_dir}")


Training and evaluating YOLOv8m...
New https://pypi.org/project/ultralytics/8.3.140 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.123  Python-3.12.1 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=./Dataset\YOLODatasetFull\dataset.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=YOLOv8m_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, ret

train: Scanning C:\Users\Pongo\OneDrive\Documents\~Cornel\~Ideas n Innovation\Project\25-4-22 -- Parkinson Unika\Dataset\YOLODatasetFull\labels\train.cache... 9431 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9431/9431 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 346.149.8 MB/s, size: 257.7 KB)


val: Scanning C:\Users\Pongo\OneDrive\Documents\~Cornel\~Ideas n Innovation\Project\25-4-22 -- Parkinson Unika\Dataset\YOLODatasetFull\labels\val.cache... 2355 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2355/2355 [00:00<?, ?it/s]


Plotting labels to runs\detect\YOLOv8m_detection\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to runs\detect\YOLOv8m_detection
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      2.56G    0.01139    0.04341     0.8958          7        512: 100%|██████████| 1179/1179 [06:44<00:00,  2.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 148/148 [00:41<00:00,  3.53it/s]

                   all       2355       2355      0.944      0.944      0.951      0.951



50 epochs completed in 6.266 hours.
Optimizer stripped from runs\detect\YOLOv8m_detection\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\YOLOv8m_detection\weights\best.pt, 52.0MB

Validating runs\detect\YOLOv8m_detection\weights\best.pt...
Ultralytics 8.3.123  Python-3.12.1 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
Model summary (fused): 92 layers, 25,842,076 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 148/148 [00:40<00:00,  3.69it/s]


                   all       2355       2355      0.928      0.931      0.965      0.965
        healthy spiral        574        574        0.9      0.918      0.937      0.936
          healthy wave        591        591      0.955      0.937      0.983      0.982
      parkinson spiral        608        608      0.915      0.907      0.963      0.963
        parkinson wave        582        582       0.94      0.962      0.977      0.977
Speed: 0.2ms preprocess, 12.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\YOLOv8m_detection
Model and results saved to: ./results


In [9]:
import shutil

source_dir = f"runs/detect/"
destination_dir = os.path.join(results_dirs, model_name)

# Move the directory
if os.path.exists(source_dir):
    shutil.move(source_dir, destination_dir)
    print(f"Results successfully moved to: {destination_dir}")
else:
    print(f"Source directory not found: {source_dir}")

Results successfully moved to: ./results\2025-05-20_21-54-25, YOLOv8m with E-B 50-8\YOLOv8m


In [10]:
import os
import numpy as np
from datetime import datetime

# Define base local path
local_base_path = os.getcwd()  # or set your own path: e.g., "D:/YOLOProject"
results_dirs = os.path.join(local_base_path, "results")
os.makedirs(results_dirs, exist_ok=True)

# Validate the model
val_results = model.val()

# Create a subfolder for metrics
output_dir = os.path.join(results_dirs, "Metrics")
os.makedirs(output_dir, exist_ok=True)

# Collect metrics
metrics = {
    "AP50": val_results.box.map50,         # Mean AP at IoU=0.50
    "AP": val_results.box.map,            # Mean AP at IoU=0.50:0.95
    "Precision": val_results.box.mp,      # Mean Precision
    "Recall": val_results.box.mr,         # Mean Recall
    "F1-Score": val_results.box.f1,       # F1 score (list per class, optional)
}

# Create timestamp and model name manually if not already defined
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
date_now = datetime.now().strftime("%Y-%m-%d")
# model_name = model.model.cfg if hasattr(model.model, 'cfg') else "yolov8_model"

# Save metrics to a file with header
metrics_file_path = os.path.join(output_dir, f"metrics_{model_name}_{current_time}.txt")
with open(metrics_file_path, "w") as f:
    f.write(f"Model: {model_name}\n")
    f.write(f"Validation Time: {current_time.replace('_', ' ')}\n\n")
    f.write(f"Training Duration: {training_duration_hours:.3f} hours for {epoch} epochs\n\n")  # <-- This line

    for metric, value in metrics.items():
        if isinstance(value, (list, np.ndarray)):
            value_str = ", ".join(f"{v:.4f}" for v in value)
            line = f"{metric}: [{value_str}]"
        else:
            line = f"{metric}: {value:.4f}"
        f.write(line + "\n")

# Display metrics in console
print("\nMetrics:")
with open(metrics_file_path, "r") as f:
    print(f.read())

print(f"Metrics saved to {metrics_file_path}")

# Save validation plots manually
results_dir = val_results.save_dir
if os.path.exists(results_dir):
    destination_dir = os.path.join(results_dirs, "Validation_Plots")
    os.makedirs(destination_dir, exist_ok=True)
    os.system(f"cp -r \"{results_dir}\" \"{destination_dir}\"")
    print(f"Validation plots saved to {destination_dir}")
else:
    print("Validation results directory not found.")


Ultralytics 8.3.123  Python-3.12.1 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
Model summary (fused): 92 layers, 25,842,076 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 1382.8378.5 MB/s, size: 211.6 KB)


val: Scanning C:\Users\Pongo\OneDrive\Documents\~Cornel\~Ideas n Innovation\Project\25-4-22 -- Parkinson Unika\Dataset\YOLODatasetFull\labels\val.cache... 2355 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2355/2355 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 295/295 [01:09<00:00,  4.26it/s]


                   all       2355       2355      0.927      0.931      0.965      0.965
        healthy spiral        574        574        0.9      0.918      0.937      0.937
          healthy wave        591        591      0.954      0.937      0.983      0.982
      parkinson spiral        608        608      0.915      0.907      0.963      0.963
        parkinson wave        582        582      0.939      0.962      0.977      0.977
Speed: 0.3ms preprocess, 25.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\YOLOv8m_detection

Metrics:
Model: YOLOv8m
Validation Time: 2025-05-21 04-14-50

Training Duration: 6.312 hours for 50 epochs

AP50: 0.9649
AP: 0.9647
Precision: 0.9268
Recall: 0.9312
F1-Score: [0.9089, 0.9454, 0.9112, 0.9502]

Metrics saved to c:\Users\Pongo\OneDrive\Documents\~Cornel\~Ideas n Innovation\Project\25-4-22 -- Parkinson Unika\results\Metrics\metrics_YOLOv8m_2025-05-21_04-14-50.txt
Validation plots saved to c:\Users\Pongo\OneD